<a href="https://colab.research.google.com/github/ixomaxip/nasoc/blob/master/src/HASOC2021_finetunedBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import os
import datetime
import re

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# Get and prepare data

In [2]:
#constants and parameters
seed = 42

path_gdrive_saved = '/content/drive/My Drive/nasoc/bert_tuned.weights'

batch_size = 32

DOWNLOAD_DATASET = False

In [3]:
from google.colab import drive
drive.mount('/content/drive')

path_gdrive = '/content/drive/My Drive/nasoc/en_Hasoc2021_train.csv'
#with open(path_gdrive, 'r') as f:
#  print(f.read())

pd.read_csv(path_gdrive)

Mounted at /content/drive


,Unnamed: 0,_id,text,task_1,task_2
0,4986,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF,PRFN
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,OFFN
2,1310,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT,NONE
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,OFFN
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,OFFN
...,...,...,...,...,...
3838,1661,60c5d6bf5659ea5e55defd57,@BBCNews Let the dog deal with the wanker once...,HOF,PRFN
3839,194,60c5d6bf5659ea5e55def185,India has suffered a lot. That Chinese bastard...,HOF,HATE
3840,3988,60c5d6bf5659ea5e55def78c,People didn't give 300+ seats majority to BJP ...,HOF,HATE
3841,4212,60c5d6bf5659ea5e55defb04,"@KanganaTeam This is such a vile, xenophobic a...",HOF,PRFN


In [4]:
!wget https://hasocfire.github.io/hasoc/2021/files/en_Hasoc2021_train.zip
!unzip -P hasoc2021_en en_Hasoc2021_train.zip
!ls

--2021-08-04 14:50:55--  https://hasocfire.github.io/hasoc/2021/files/en_Hasoc2021_train.zip
Resolving hasocfire.github.io (hasocfire.github.io)... 185.199.109.153, 185.199.110.153, 185.199.111.153, ...
Connecting to hasocfire.github.io (hasocfire.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 330206 (322K) [application/zip]
Saving to: ‘en_Hasoc2021_train.zip’

en_Hasoc2021_train. 100%[===================>] 322.47K  --.-KB/s    in 0.04s   

2021-08-04 14:50:56 (7.41 MB/s) - ‘en_Hasoc2021_train.zip’ saved [330206/330206]

Archive:  en_Hasoc2021_train.zip
  inflating: en_Hasoc2021_train.csv  
drive  en_Hasoc2021_train.csv  en_Hasoc2021_train.zip  sample_data


In [6]:
raw_dataset = pd.read_csv('en_Hasoc2021_train.csv')

raw_dataset['inputs'] = pd.Categorical(raw_dataset['task_1'])
raw_dataset['inputs'] = raw_dataset.inputs.cat.codes

raw_dataset['task_2_cat'] = pd.Categorical(raw_dataset['task_2'])
raw_dataset['task_2_cat'] = raw_dataset.task_2_cat.cat.codes

print(list(raw_dataset.columns.values.tolist()))
print(raw_dataset.head())

train_df, test_df = train_test_split(raw_dataset, test_size=0.2, random_state=seed)

sr_train = train_df.groupby('task_1').count()['text']
sr_test  = test_df.groupby('task_1').count()['text']

tr_pr = sr_train['NOT']/sr_train['HOF']
ts_pr = sr_test['NOT']/sr_test['HOF']

print(f"Train set: {sr_train}\n\nTest set: {sr_test}\n\nNeutral to offensive proportion in the train set: {tr_pr:.2f} and in the test set: {ts_pr:.2f}")

['Unnamed: 0', '_id', 'text', 'task_1', 'task_2', 'inputs', 'task_2_cat']
   Unnamed: 0                       _id  ... inputs task_2_cat
0        4986  60c5d6bf5659ea5e55defa2c  ...      0          3
1        3394  60c5d6bf5659ea5e55def461  ...      0          2
2        1310  60c5d6bf5659ea5e55defaad  ...      1          1
3        3390  60c5d6bf5659ea5e55def419  ...      0          2
4        4626  60c5d6bf5659ea5e55def7fa  ...      0          2

[5 rows x 7 columns]
Train set: task_1
HOF    1972
NOT    1102
Name: text, dtype: int64

Test set: task_1
HOF    529
NOT    240
Name: text, dtype: int64

Neutral to offensive proportion in the train set: 0.56 and in the test set: 0.45


In [7]:
#Optional. Save the files
from google.colab import files

if DOWNLOAD_DATASET:
    train_df.to_csv('train.csv', index=False)
    test_df.to_csv('validate.csv', index=False)

    files.download('train.csv')
    files.download('validate.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#copied from https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

# copied from https://stackoverflow.com/questions/64719706/cleaning-twitter-data-pandas-python
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    #tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(tweet.split())
    tweet = remove_emojis(tweet)
    #tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
    #     if w.lower() in words or not w.isalpha())
    return tweet


train_df, test_df = train_test_split(raw_dataset, test_size=0.2, random_state=seed)
text = train_df.pop('text').map(lambda line: cleaner(line))
label = train_df.pop('inputs')

text_val = test_df.pop('text').map(lambda line: cleaner(line))
label_val = test_df.pop('inputs')

print(text[:5])

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((text.values, label.values))
val_ds = tf.data.Dataset.from_tensor_slices((text_val.values, label_val.values))

train_ds = train_ds.batch(batch_size=batch_size, drop_remainder=True)
val_ds = val_ds.batch(batch_size=batch_size, drop_remainder=True)

# Model preparation

In [ ]:
!pip install -q -U tensorflow-text
!pip install -q tf-models-official